In [1]:
import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  version='20200918'
  build='nightly'
  urlretrieve(f"https://drake-packages.csail.mit.edu/drake/{build}/drake-{version}/setup_drake_colab.py",
              "setup_drake_colab.py")
  from setup_drake_colab import setup_drake
  setup_drake(version=version, build=build)
  !pip install pyngrok

# Install pyngrok.
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Imports
import numpy as np
import pydot
from ipywidgets import Dropdown, Layout
from IPython.display import display, HTML, SVG

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback


  Created wheel for pyngrok: filename=pyngrok-4.2.2-cp36-none-any.whl size=18025 sha256=f069f29eeba534e2903fd7359d112a792a214847c5fd81a3cc331ecd7699e48b
  Stored in directory: /root/.cache/pip/wheels/9c/d3/b5/502fbef4b2782a6142bc00c62ccde76ee04c9148fb43b1b2cd
Successfully built pyngrok


In [31]:
positions =  [np.pi/2]*7

plant = MultibodyPlant(time_step=1e-4)
Parser(plant).AddModelFromFile(FindResourceOrThrow("drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf"))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_link_0"))
plant.Finalize()
context = plant.CreateDefaultContext()

plant.SetPositions(context,positions)

M=plant.CalcMassMatrixViaInverseDynamics(context)
np.set_printoptions(suppress=True) 
np.set_printoptions(precision=4)

print(M)

[[ 3.0783  0.0408 -0.0363 -0.787  -0.0488  0.0514  0.001 ]
 [ 0.0408  2.3226  1.0043 -0.0408 -0.0547  0.     -0.    ]
 [-0.0363  1.0043  0.7906 -0.     -0.0189 -0.0168  0.    ]
 [-0.787  -0.0408 -0.      0.7565  0.0488  0.     -0.001 ]
 [-0.0488 -0.0547 -0.0189  0.0488  0.0242 -0.      0.    ]
 [ 0.0514  0.     -0.0168  0.     -0.      0.0168  0.    ]
 [ 0.001  -0.      0.     -0.001   0.      0.      0.001 ]]
